# ICESat-2 ATL15 Viewer
Interactive visualization and data extraction tool for ICESat-2 Gridded Land Ice Height Change data (ATL15)

In [ ]:
import IS2view
# autoreload
%load_ext autoreload
%autoreload 2

#### Set parameters for ATL15
- Asset: Location to get the data
- Directory: Working data directory
- Release: ATL15 data release (001, 002, 003, 004)
- Region: ATL15 data region (AA, A1, A2, A3, A4, CN, CS, GL, IS, RA, SV)
- Resolution: ATL15 horizontal resolution (01km, 10km, 20km, 40km)
- Group: ATL15 data group to read from file
- Format: ATL15 data format to read (nc, zarr)

In [ ]:
IS2widgets = IS2view.widgets()
IS2widgets.VBox([
    IS2widgets.asset,
    IS2widgets.directory,
    IS2widgets.release,
    IS2widgets.region,
    IS2widgets.resolution,
    IS2widgets.group,
    IS2widgets.format,
])

### Query CMR for ATL15 granule
NASA's Common Metadata Repository (CMR) is a catalog of metadata records contained within the Earth Observing System Data and Information System (EOSDIS). ICESat-2 ATL15 data is archived at the NASA Distributed Active Archive Center (DAAC) at the [National Snow and Ice Data Center (NSIDC)](https://nsidc.org/daac/).

In [ ]:
granule = IS2view.utilities.query_resources(
    asset=IS2widgets.asset.value,
    directory=IS2widgets.directory.value,
    product='ATL15',
    release=IS2widgets.release.value,
    region=IS2widgets.region.value,
    resolution=IS2widgets.resolution.value,
    format=IS2widgets.format.value,
)

### Read and inspect ATL15 data
The selected group within ATL15 data will be read using [xarray](https://xarray.dev/) and [rioxarray](https://corteva.github.io/rioxarray/).

In [ ]:
ds = IS2view.open_dataset(granule,
    group=IS2widgets.group.value,
    format=IS2widgets.format.value)
ds

### Interactive Mapping with Leaflet

Interactive maps within IS2view are built upon [ipyleaflet](https://ipyleaflet.readthedocs.io). Clicking and dragging will pan the field of view, and zooming will adjust the field of view. There are 2 polar stereographic projections available for mapping in IS2view ([North](https://epsg.io/3413) and [South](https://epsg.io/3031)). The map projection, map center and zoom level will all be set based on the ATL15 region selected. The available basemaps are NASA's Next Generation Blue Marble visualization for the Arctic and Antarctic regions.

Regional time series can be extracted from ATL15 by interactively drawing geometries on the leaflet map or programmatically using [geopandas GeoDataFrames](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html). See [Recipes](https://is2view.readthedocs.io/en/latest/user_guide/Recipes.html) for more information.

#### Set plot parameters for ATL15
Specifies the variable to plot, the [colormap](https://matplotlib.org/gallery/color/colormap_reference.html), and the normalization for the plot colors.

In [ ]:
# create leaflet map
m = IS2view.Leaflet(IS2widgets.projection,
    center=IS2widgets.center,
    zoom=IS2widgets.zoom,
    draw_control=True,
    attribution=False)
# set plot attributes
IS2widgets.get_variables(ds)
IS2widgets.set_atl15_defaults()
wbox = IS2widgets.VBox([
    IS2widgets.variable,
    IS2widgets.timestep,
    IS2widgets.dynamic,
    IS2widgets.range,
    IS2widgets.cmap,
    IS2widgets.reverse,
])
# display as a horizontal row
IS2widgets.HBox([m.map, wbox])

### Add xarray dataset as an image service layer

In [ ]:
ds.leaflet.plot(m.map, lag=IS2widgets.lag,
    vmin=IS2widgets.vmin, vmax=IS2widgets.vmax,
    group=IS2widgets.group.value,
    variable=IS2widgets.variable.value,
    cmap=IS2widgets.colormap,
    opacity=0.75,
    enable_popups=False)
# observe changes in widget parameters
ds.leaflet.set_observables(IS2widgets)

### Extract time series from selected geometries
- Point: time series for a geolocation
- Transect: time series for a line segment
- Regional average: time series for a polygon or bounding box

In [ ]:
for feature in m.geometries['features']:
    ds.timeseries.plot(feature,
        variable=IS2widgets.variable.value,
    )